In [86]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:

def create_bedrock_llm(model_version_id):
    bedrock_client = boto3.client('bedrock-runtime', region_name='eu-central-1')
    bedrock_llm = Bedrock(
        model_id=model_version_id, 
        client=bedrock_client,
        model_kwargs={'temperature': 0}
        )
    return bedrock_llm

# First

In [16]:
# text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=20)
# documents = text_splitter.split_documents(docs)

vectordb = Chroma.from_documents(
  docs,
  embedding=OpenAIEmbeddings(),
  persist_directory='../data/db_chroma'
)
vectordb.persist()

In [92]:
qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=False
)
# def pre_analysis():
#     qa_chain = RetrievalQA.from_chain_type(
#         llm=llm_options[st.session_state.selected_model],
#         retriever=db_pers.as_retriever(search_kwargs={'k': 7}),
#         return_source_documents=False
#         )
#     response = qa_chain({'query': prompt})
#     print(json.loads(response['result']))
prompt = """\
Extract relevant information from these experiences. It should return a json containing:
- job title
- company
- start date
- end date
- location
- company details
- relevant skills
"""
result = qa_chain({'query': prompt})
result

NameError: name 'ChatOpenAI' is not defined

In [9]:
from langchain.document_loaders import HuggingFaceDatasetLoader
dataset_name = "frtna/FineTunetaxonomy"
page_content_column = "combined"
loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
data = loader.load()

/Users/hamit.kavas/Library/Caches/pypoetry/virtualenvs/ml-cross-bedrock-user-guide-aldTT5cv-py3.11/lib/python3.11/site-packages/datasets/load.py:2089: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=None' instead.
  warnings.warn(


In [10]:
ESCO_data = data[:3015]
len(ESCO_data)

3015

In [ ]:
# from chromadb import Documents, EmbeddingFunction, Embeddings
# import cohere
# from dotenv import load_dotenv
# load_dotenv()
# COHERE_API_KEY = os.getenv("COHERE_KEY")
# co = cohere.Client(COHERE_API_KEY)
# class CohereEmbeddingFunction(EmbeddingFunction):
#     def __call__(self, texts: Documents) -> Embeddings:
#         # embed the documents somehow
#         return co.embed(texts, model="embed-multilingual-v3.0", input_type='search_document').embeddings
#     def embed_documents(self, texts: Documents) -> Embeddings:
#         # embed the documents somehow
#         embeddings =  co.embed(texts, model="embed-multilingual-v3.0", input_type='search_document').embeddings
#         return [list(map(float, e)) for e in embeddings]
#     def embed_query(self, texts: Documents) -> Embeddings:
#         # embed the documents somehow
#         embeddings = co.embed([texts], model="embed-multilingual-v3.0", input_type='search_query').embeddings
#         return [list(map(float, e)) for e in embeddings][0]


# cohere_ef = CohereEmbeddingFunction()

# # import chromadb
# # from chromadb.config import Settings
# # client = chromadb.PersistentClient(path="../text-embeddings-llm-inference/cohere_multilingual_embeddings.db", 
# #     settings=Settings(anonymized_telemetry=False, allow_reset=False))
 
# from langchain.document_loaders import HuggingFaceDatasetLoader
# dataset_name = "frtna/FineTunetaxonomy"
# page_content_column = "combined"
# loader = HuggingFaceDatasetLoader(dataset_name, page_content_column)
# data = loader.load()

# import time
# for ix in range(0, len(data), 100):
#     # Process the current batch of documents.
#     batch = data[ix:ix+100]
#     # Create vector embeddings for the current batch.
#     vectordb = Chroma.from_documents(batch, embedding=cohere_ef, persist_directory='cohere_embeddings_db')
#     # Persist the current batch.
#     vectordb.persist()
#     # Wait for 60 seconds before processing the next batch.
#     time.sleep(80)

# db_pers = Chroma(persist_directory="../text-embeddings-llm-inference/cohere_multilingual_embeddings.db", embedding_function=cohere_ef)#, client=client)
# db_pers.persist()


# query = "Industrial engineering but more with design and less with management"
# docs = db_pers.similarity_search_with_score(query)
# docs

In [11]:
from sentence_transformers import SentenceTransformer
from chromadb import Embeddings
from langchain.vectorstores import Chroma
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
# sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=MODEL_NAME, device='cpu')
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self._embedding_function = SentenceTransformer(model_name)
    def embed_documents(self, texts):
        embeddings = self._embedding_function.encode(texts, convert_to_numpy=True).tolist()
        return [list(map(float, e)) for e in embeddings]
    def embed_query(self, text):
        embeddings = self._embedding_function.encode([text], convert_to_numpy=True).tolist()
        return [list(map(float, e)) for e in embeddings][0]

sentence_transformer_ef = SentenceTransformerEmbeddings(model_name=MODEL_NAME)
# vectordb = Chroma.from_documents(ESCO_data, embedding=sentence_transformer_ef, persist_directory='ESCO_mpnet_db')
# vectordb.persist()

In [56]:
# Define the number of query results to retrieve.
job_offer = 'Job Title: Stone Masonary'
NUM_QUERY = 10
vectordb = Chroma(persist_directory="ESCO_mpnet_db", embedding_function=sentence_transformer_ef)
docs = vectordb.similarity_search(job_offer, NUM_QUERY)
docs

[Document(page_content="Job Title: stonemason; Description: Stonemasons manually carve and assemble stone for construction purposes. While CNC operated carving equipment is the industry standard, artisanal carving for ornamental stone is still done manually.; Synonyms: architectural stone mason, architectural stonemason, artisanal stonemason, artisanal stone mason, banker mason, building mason, craft mason, craft stonemason, fixer mason, heritage stone mason, heritage stonemason, mason, memorial mason, memorial stonemason, stone carver, stone cutter, stone finisher, stone mason, stonemason, stone setter, stoneworker, structural stoneworker, tombstone carver; Skills: create cutting plan, follow health and safety procedures in construction, inspect construction supplies, inspect stone surface, interpret 2D plans, interpret 3D plans, maintain work area cleanliness, mark stone workpieces, operate grinding hand tools, polish stone by hand, prepare stone for smoothing, regulate cutting speed

In [57]:
# Define the number of query results to retrieve.
job_offer = 'Stone Masonary'
NUM_QUERY = 10
vectordb = Chroma(persist_directory="ESCO_mpnet_db", embedding_function=sentence_transformer_ef)
docs = vectordb.similarity_search(job_offer, NUM_QUERY)
docs

[Document(page_content="Job Title: stonemason; Description: Stonemasons manually carve and assemble stone for construction purposes. While CNC operated carving equipment is the industry standard, artisanal carving for ornamental stone is still done manually.; Synonyms: architectural stone mason, architectural stonemason, artisanal stonemason, artisanal stone mason, banker mason, building mason, craft mason, craft stonemason, fixer mason, heritage stone mason, heritage stonemason, mason, memorial mason, memorial stonemason, stone carver, stone cutter, stone finisher, stone mason, stonemason, stone setter, stoneworker, structural stoneworker, tombstone carver; Skills: create cutting plan, follow health and safety procedures in construction, inspect construction supplies, inspect stone surface, interpret 2D plans, interpret 3D plans, maintain work area cleanliness, mark stone workpieces, operate grinding hand tools, polish stone by hand, prepare stone for smoothing, regulate cutting speed

In [7]:
print("There are", vectordb._collection.count(), "in the collection")

There are 0 in the collection


In [95]:
docs[0].page_content

"Job Title: financial auditor; Description: Financial auditors collect and examine financial data for clients, organisations and companies. They ensure the financial data is properly maintained and free of material misstatements due to error or fraud, that it adds up, and functions legally and effectively. They review lending and credit policies or numbers in databases and documents, evaluate, consult and assist the source of the transaction if necessary. They use their review of the client's financial governance as assurance to give testimony to the shareholders, stakeholders and board of directors of the organisation or company that all is up to par.; Synonyms: audit manager, auditor, bank auditor, bank examiner, chartered accountant, external auditor, external examiner, finance auditor, finances auditor, financial auditor, financial examiner, financial systems analyst, internal auditor, internal examiner, senior internal auditor; Skills: analyse financial risk, arrange audit, check 

In [49]:
from langchain.llms.bedrock import Bedrock
from langchain.embeddings import BedrockEmbeddings
import boto3
bedrock_client = boto3.client('bedrock-runtime', region_name='eu-central-1')
def create_langchain_vector_embedding_using_bedrock(bedrock_client, bedrock_embedding_model_id):
    bedrock_embeddings_client = BedrockEmbeddings(
        client=bedrock_client,
        model_id=bedrock_embedding_model_id)
    return bedrock_embeddings_client
    

def create_bedrock_llm(bedrock_client, model_version_id):
    bedrock_llm = Bedrock(
        model_id=model_version_id, 
        client=bedrock_client,
        model_kwargs={'temperature': 0}
        )
    return bedrock_llm

llm = create_bedrock_llm(bedrock_client=bedrock_client, model_version_id="anthropic.claude-v2")
embeddings = create_langchain_vector_embedding_using_bedrock(bedrock_client=bedrock_client, bedrock_embedding_model_id="anthropic.claude-v2")


In [53]:
# import os
# from langchain.document_loaders import PyPDFLoader

# pdf_loader = PyPDFLoader('../data/example_cv_2.pdf', extract_images=True)
# docs = pdf_loader.load()
# def temp_vectorstore(docs):
#     vectordb = Chroma.from_documents(
#     docs,
#     embedding=sentence_transformer_ef,
#     persist_directory='../data/temp_db'
#     )
#     vectordb.persist()
#     return vectordb
# temp_vectordb = temp_vectorstore(docs)

In [130]:
context

['Job Title: Auditor; Description: We are seeking a dynamic individual to be a part of our Audit department. As our auditor, your responsibilities will include planning and performing a variety of internal audits and special projects throughout the company, which include a variety of Senior Management driven assignments.These audits will include detailed assessment of operational, financial and systems activities where a thorough understanding of control processes must be obtained from a variety of people and resources. You&rsquo;ll provide recommendations that will include strengthening internal controls, mitigating financial risks and safeguarding the corporation.; Synonyms: ; Skills: - 3 to 5 years of experience auditing large companies in either Public Accounting or industry.  - Strong technical knowledge of US generally accepted accounting principles.- A CPA or CIA is preferred.- Excellent communication skills (written and oral) with a strong attention to detail.  - Ability to wor

In [136]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
job_offer = 'We are seeking a dynamic individual to be a part of our Audit department. As our auditor, your responsi- bilities will include planning and performing a variety of internal audits and special projects throughout the company, which include a variety of Senior Manage- ment driven assignments. These audits will include detailed assessment of operational, financial and systems activities where a thorough understanding of control processes must be obtained from a vari- ety of people and resources.'
qry = '''**Please list generally required skills by different job offers that closely matches the given Job Offer:**
**Job Offer Details**
{JobOffer}
'''
input_query = qry.format(JobOffer=job_offer)
context = vectordb.similarity_search(job_offer, NUM_QUERY)
input_data = {"input_documents": context, "question": input_query}
output = chain(input_data)
print(output['output_text'])

" Based on the job descriptions provided, here are some generally required skills for the auditor position that match the given job offer:\n\n- 2-5 years of auditing experience, including operational, financial, and systems auditing\n- Knowledge of accounting principles and standards (GAAP, SOX, etc.)  \n- Strong analytical and problem solving skills\n- Ability to evaluate internal controls and processes\n- Excellent written and verbal communication skills\n- Detail-oriented with strong organizational skills  \n- Proficient with Microsoft Office suite, including Excel\n- Bachelor's degree in Accounting, Finance or related field\n- CPA or other auditing certification preferred\n- Ability to work independently and meet deadlines \n- Experience working across different functions and management levels\n- Some travel required (up to 25-30%)"

In [137]:
print(output['output_text'])

 Based on the job descriptions provided, here are some generally required skills for the auditor position that match the given job offer:

- 2-5 years of auditing experience, including operational, financial, and systems auditing
- Knowledge of accounting principles and standards (GAAP, SOX, etc.)  
- Strong analytical and problem solving skills
- Ability to evaluate internal controls and processes
- Excellent written and verbal communication skills
- Detail-oriented with strong organizational skills  
- Proficient with Microsoft Office suite, including Excel
- Bachelor's degree in Accounting, Finance or related field
- CPA or other auditing certification preferred
- Ability to work independently and meet deadlines 
- Experience working across different functions and management levels
- Some travel required (up to 25-30%)


In [13]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
job_offer = 'We are seeking a dynamic individual to be a part of our Audit department. As our auditor, your responsi- bilities will include planning and performing a variety of internal audits and special projects throughout the company, which include a variety of Senior Manage- ment driven assignments. These audits will include detailed assessment of operational, financial and systems activities where a thorough understanding of control processes must be obtained from a vari- ety of people and resources.'
prompt_ = """    
        Human: Use the following pieces of context to answer the question.

        {context}
                
        Job Offer: {job_offer}
        Question: what are the required skills for this Job Offer?
        Helpful Answer:
        """
context = [item.page_content for item in vectordb.similarity_search(query, NUM_QUERY)]
qa_chain = RetrievalQA.from_chain_type(
        llm=ChatOpenAI(),
        retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
        return_source_documents=True
        )
out=qa_chain({'query': prompt_, 'job_offer':job_offer, 'context':context})
out['result']

"I'm sorry, but you haven't provided any context or a specific job offer for me to use in order to answer your question. Can you please provide more information?"

In [14]:
context

['Job Title: online marketer; Description: Online marketers use e-mail, internet and social media in order to market goods and brands.; Synonyms: assistant online marketer, digital marketer, e-commerce marketer, e-commerce marketing specialist, graduate online marketer, internet marketer, internet marketing specialist, online marketer, online marketing specialist, online sales channel marketer, senior online marketer, social media marketing coordinator, social media marketing specialist, trainee online marketer; Skills: apply customer engagement strategy, apply social media marketing, apply strategic thinking, conduct mobile marketing, create new concepts, execute conversion testing, execute email marketing, implement marketing strategies, implement sales strategies, inspect data, manage budgets, perform copywriting, perform image editing, perform market research, perform online data analysis, perform project management, perform video editing, plan digital marketing, use content manage

In [32]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

# we are specifying that OpenAI is the LLM that we want to use in our chain
query = """    
        Use the following pieces of context to answer the question regarding the given CV.

        CV: {cv} // i.e the CV we want to extract information from

        {context} // i.e the chunks of ESCO text retrieved deemed to be moset semantically
                // relevant to our question

        Question: {query} // i.e our actualy query
        Helpful Answer:
        """

from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

qa_chain = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectordb.as_retriever(search_kwargs={'k': 7}),
    return_source_documents=True
)

# we can now execute queries against our Q&A chain
# result = qa_chain({'query': 'Who is the CV about?'})
result = qa_chain({'query': 'What are the best fit job title for this CV based on ESCO taxonomy?', 'cv':cv})

print(result['result'])

ValidationError: 1 validation error for OpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass  `openai_api_key` as a named parameter. (type=value_error)

In [41]:
cv = {
"job title":"sport facility manager",
"company":"Fragadis",
"start date":"07/18",
"end date":"09/19",
"location":"Vic",
"company details":"- Amplia experiencia en ayudar a los clientes. Buen manejo de la caja registradora y del cobro en efectivo y con tarjeta de crédito.",
"relevant skills":"Ayudar a los clientes, manejo de caja registradora, cobro en efectivo y con tarjeta de crédito"
}

In [47]:
from langchain.chains import ChatVectorDBChain
pdf_qa = ChatVectorDBChain.from_llm(OpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
                                    vectordb, return_source_documents=True)
query = 'I have a business in digital marketing. Does she have compatible skills?'
result = pdf_qa({"question": query, "chat_history": ""})
print("Answer:")
print(result["answer"])

/Users/hamit.kavas/Library/Caches/pypoetry/virtualenvs/ml-cross-bedrock-user-guide-aldTT5cv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/hamit.kavas/Library/Caches/pypoetry/virtualenvs/ml-cross-bedrock-user-guide-aldTT5cv-py3.11/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/Users/hamit.kavas/Library/Caches/pypoetry/virtualenvs/ml-cross-bedrock-user-guide-aldTT5cv-py3.11/lib/python3.11/site-packages/langchain/chains/conversational_retrieval/base.py:397: UserWarning: `ChatVectorDBChain` is deprecated - please use `from langchain.chains import ConversationalRetrievalChain`
  warning

Answer:
Yes, Lauren Chen has compatible skills for a business in digital marketing. She has 4+ years of experience in online marketing, branding, and business strategy, as well as experience in managing digital sales and streaming accounts, developing strategic partnerships, and executing advertising campaigns. Additionally, she has skills in digital data analytics, Adobe Photoshop, Adobe Illustrator, Adobe InDesign, AutoCAD, and Microsoft Office.


In [37]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0), vectordb.as_retriever(search_kwargs={"k": 8}), memory=memory)

In [49]:
# result = qa({"question": "what was she working for in 2018? Can you give me a description of her role?"})
# result = qa({"question": "what are her skills?"})
result = qa({"question": "but my bussiness is more about sofware development. Can she still be a good candidate?"})
result

Number of requested results 8 is greater than number of elements in index 2, updating n_results = 2


{'question': 'but my bussiness is more about sofware development. Can she still be a good candidate?',
 'chat_history': [HumanMessage(content='what was she working for in 2018? Can you give me a description of her role?'),
  AIMessage(content=' Lauren Chen was working as a Digital Marketing Associate for Momo Software in 2018. In this role, she worked with management to develop and apply digital marketing plans with a focus on driving acquisition and conversion. She also devised and implemented robust digital acquisition plans, ensuring precision in financial reporting, budgets, and forecasts.'),
  HumanMessage(content='what are her skills?'),
  AIMessage(content=' Lauren Chen has skills in digital data analytics, digital marketing, Adobe Photoshop, Adobe Illustrator, Adobe InDesign, AutoCAD, Rhinoceros, Microsoft Office, and Semrush, Ahrefs, and Page Optimizer Pro.'),
  HumanMessage(content='what is her phone number?'),
  AIMessage(content=' (212) 256-1414'),
  HumanMessage(content='w

In [61]:
import base64
import requests

# OpenAI API Key
api_key = OPENAI_API_KEY

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Path to your image
image_path = "../data/example.jpg"

# Getting the base64 string
base64_image = encode_image(image_path)

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {api_key}"
}

payload = {
    "model": "gpt-4-vision-preview",
    "messages": [
      {
        "role": "user",
        "content": [
          {
            "type": "text",
            "text": "What’s in this image?"
          },
          {
            "type": "image_url",
            "image_url": {
              "url": f"data:image/jpeg;base64,{base64_image}"
            }
          }
        ]
      }
    ],
    "max_tokens": 300
}

response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

print(response.json())

{'id': 'chatcmpl-8IFhhEvlexGdzdQsPyRMudMBkSNSS', 'object': 'chat.completion', 'created': 1699360961, 'model': 'gpt-4-1106-vision-preview', 'usage': {'prompt_tokens': 778, 'completion_tokens': 283, 'total_tokens': 1061}, 'choices': [{'message': {'role': 'assistant', 'content': "The image is a resume or curriculum vitae (CV). It belongs to a person named Lauren Chen, who is advertising herself as a Digital Marketing Specialist. The resume is structured with sections for Career Objective, Professional Experience, Contact, Education, and Relevant Skills.\n\nIn the Career Objective, Lauren discusses her 4+ years of experience in online marketing, branding, and business strategy, mainly in music, media, and entertainment industries. She highlights her skills in driving revenue and brand growth through digital strategies.\n\nProfessional Experience lists her roles, including her current position at Triangle Music Group in New York as a Digital Marketing Specialist since September 2019, her pa

In [66]:
response.json()['choices'][0]['message']['content']

"The image is a resume or curriculum vitae (CV). It belongs to a person named Lauren Chen, who is advertising herself as a Digital Marketing Specialist. The resume is structured with sections for Career Objective, Professional Experience, Contact, Education, and Relevant Skills.\n\nIn the Career Objective, Lauren discusses her 4+ years of experience in online marketing, branding, and business strategy, mainly in music, media, and entertainment industries. She highlights her skills in driving revenue and brand growth through digital strategies.\n\nProfessional Experience lists her roles, including her current position at Triangle Music Group in New York as a Digital Marketing Specialist since September 2019, her past role at Momo Software as a Digital Marketing Associate, and an internship at Kingston Digital.\n\nHer Contact information includes a phone number, an email address, and a LinkedIn profile link.\n\nLauren's Education credentials show that she graduated from New York Universi

# XAVI

In [247]:
import yaml

# The prompt template as a Python dictionary
prompt_template_dict = {
    "cv-parsing-prompt": """
        You are an expert in human resources and your job is to extract relevant information from curriculums.
        For each experience in the curriculum, you should return a json containing the following information:
        - Experiences: List of experiences containing relevant fields.
            - job_title: Associated job title
            - description: Description of the role
            - skills: List of skills mentioned
        - Languages: List of languages known by the candidate.
        - Highlights: Brief summary of the candidate's profile.

        Here is the curriculum:
        ```
        {cv}
        ```
        The response should be only a json.
    """,
    "job-offer-processing-prompt": """
        Please use a retriever to find job offers similar to the following: "Job Offer: {joboffer}"

        In the results, I would like you to:
        1. Highlight the key responsibilities and qualifications of the given job offer and spesify. 
        2. Identify and summarize the skills and qualifications that are commonly sought in similar job offers.

        1. Job Offer Summary:
        - Title: 
        - Department: 
        - Responsibilities:

        2. Commonly Sought Skills and Qualifications:
        - Educational Background: 
        - Experience: 
        - Skills: 
        - Technical Proficiencies: 
        - Regulatory Knowledge: 
        - Certifications: 

        Note: The specific requirements and preferences for qualifications may vary between job offers.

        Please return the information in the JSON format.
        """,
    "full-report-template": """
        I have a curriculum vitae (CV) and a job offer. I would like you to compare the two and provide insights on how well the CV matches the job offer's requirements. Please use a retriever to consult the ESCO (European Skills, Competences, Qualifications and Occupations) Taxonomy to extract relevant job profile information. This will inform a more accurate comparison between the CV and the job offer.
        Be concise and objective.
        Curriculum Vitae:
        ```
        {cv}
        ```

        Job Offer:
        ```
        {job_offer}
        ```
        Instructions for the OpenAI model:
            1. Use the ESCO Taxonomy to find job profiles that correspond with the experiences and qualifications listed in the CV.
            2. Critically evaluate the relevance of each experience and qualification in the CV against the specific requirements of the job offer.
            3. Focus on the precise nature of the job offer, considering the essential qualifications, detailed responsibilities, and the level of expertise required.
            4. Provide a realistic analysis that identifies:
            - Direct matches and transferable skills between the CV and the job offer.
            - Any qualifications or experiences in the CV that are exactly what the job offer requires.
            - Areas where the CV falls short of the job offer's requirements, including lack of specific experiences, necessary qualifications, or required certifications.
            5. Avoid using expressions that directly affect the decision, are subjective and dominant. Be concrete and clear, explain your thoughts in a transparent way.

        Return the analysis in the following JSON structure:
        {{
            "ESCO Analysis": {{
                "Educational Qualifications Comparison": "",
                "Skills and Proficiencies Match": "",
                "Certifications and Regulatory Knowledge": "",
                "Job Profile Match": "",
                "Overall Summary": {{
                    "Strong Matches": "",
                    "Potential Gaps": ""
                }}
            }}
        }}

        Note: The insights should be based on the retrieved information from the ESCO Taxonomy to ensure an informed and accurate analysis.
        """
}
yaml_str = yaml.dump(prompt_template_dict)

with open('prompt_template.yaml', 'w') as file:
    yaml.dump(prompt_template_dict, file)


# with open('prompt_template.yaml', 'r') as file:
#     loaded_template = yaml.safe_load(file)
# prompt_template = loaded_template['cv-parsing-prompt']
# prompt_template

In [ ]:
import json
with open('prompt_template.json', 'w') as file:
    json.dump(prompt_template_dict, file)

## Parse CV

In [70]:
from langchain.document_loaders import PyPDFLoader
tmp_file_path = "../../data/Sefa-CV.pdf"
loader = PyPDFLoader(tmp_file_path, extract_images=True)
cv = loader.load()

In [2]:
# print(cv[0].page_content)

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [218]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers.openai_functions import JsonKeyOutputFunctionsParser
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, conlist
from typing import List

prompt_template = """
You are an expert in human resources and your job is to extract relevant information from curriculums.
For each experience in the curriculum, you should return a json containing the following information:
- Experiences: List of experiences containing relevant fields.
    - job_title: Associated job title
    - description: Description of the role
    - skills: List of skills mentioned
- Languages: List of languages known by the candidate.
- Highlights: Brief summary of the candidate's profile.

Here is the curriculum:
```
{cv}
```
The response should be only a json.
""".strip()
#  The response should be only a jsonin the following format:
# ```
# {{
#     'experiences': [
#         *json containing experience details*
#     ],
#     'languages': [
#         *list of languages*
#     ]
# }}
# ```
model = ChatOpenAI()

class ApplicantExperiences(BaseModel):
    experiences: str = Field(description="Extracted experiences from the CV")
    languages: str = Field(description="Languages known by the candidate in CV")
class ApplicantExperiencesArray(BaseModel):
    CV: List[ApplicantExperiences] = Field(description="List of experiences and lanuages")
pydantic_parser = PydanticOutputParser(pydantic_object=ApplicantExperiencesArray)
format_instructions = pydantic_parser.get_format_instructions()

# prompt = ChatPromptTemplate.from_template(prompt_template, format_instructions=format_instructions)
prompt = ChatPromptTemplate.from_template(prompt_template)
# import re
# def parse_json():
#     json_match = re.search(r'```json\n(.+)\n```', answer.content, re.DOTALL)
#     if json_match:
#         json_string = json_match.group(1)
#         # Parse the JSON string
#         json_object = json.loads(json_string)
#         return json_object
#     else:
#         error_message = "No JSON object found in the string."
#         return error_message
# json_object = parse_json()


chain = {"context": retriever, "cv": RunnablePassthrough()} | prompt | model 
answer = chain.invoke({"cv": cv})
response = json.loads(answer.content)
CV =response
response

{'Experiences': [{'job_title': 'Founder',
   'description': 'I founded a company that sells pet products and pet foods online. Our products are sold at www.zoopet.com.tr and many online marketplaces. Operations of the company involved shipment of the products to every region of Turkey. In 2022, it was taken over by an investor. The company is still inactive service today.',
   'skills': ['Ability to Work Under Pressure']},
  {'job_title': 'Sales Manager',
   'description': 'I began my career at Narven, a timeshare company in Bolu as a Sales Representative. My role involved direct customer interactions, customer relationship management activities, and conducting market analysis. After 18 successful months, I was promoted to Sales Manager, overseeing a sales team. I received multiple internal performance awards before departing to launch my own venture.',
   'skills': ['Effective Time Management',
    'Hard Working',
    'Highly Organized',
    'Fast Learner']},
  {'job_title': 'Delivery

## Run RAG on ESCO 

In [88]:
from sentence_transformers import SentenceTransformer
from chromadb import Embeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
MODEL_NAME = 'sentence-transformers/paraphrase-multilingual-mpnet-base-v2'
class SentenceTransformerEmbeddings(Embeddings):
    def __init__(self, model_name):
        self._embedding_function = SentenceTransformer(model_name)
    def embed_documents(self, texts):
        embeddings = self._embedding_function.encode(texts, convert_to_numpy=True).tolist()
        return [list(map(float, e)) for e in embeddings]
    def embed_query(self, text):
        embeddings = self._embedding_function.encode([text], convert_to_numpy=True).tolist()
        return [list(map(float, e)) for e in embeddings][0]

sentence_transformer_ef = SentenceTransformerEmbeddings(model_name=MODEL_NAME)

In [ ]:
# template = """**You are an expert in human resources and your job is extracting relevant information from Job Offer and comparing this with other Job Offers in database.**
#         Compare :
#         - Required Experiences: List of required experiences required by given job offer.
#             - job_title: Associated job title
#             - description: Description of the role
#             - skills: List of skills mentioned
#         - Highlights - Brief Summary of the Job Offer
        
#         **Similar Job Offer Details**
#         {context}
#             - job_title: Associated job title
#             - description: Description of the role
#             - skills: List of skills mentioned

#         Job Offer: {question}
#         The response should be only a string with bullets.
#         """

In [189]:
vectorstore = Chroma(persist_directory="ESCO_mpnet_db", embedding_function=sentence_transformer_ef)

retriever = vectorstore.as_retriever()

template = """
Please use a retriever to find job offers similar to the following: "Job Offer: {question}"

In the results, I would like you to:
1. Highlight the key responsibilities and qualifications of the given job offer and spesify. 
2. Identify and summarize the skills and qualifications that are commonly sought in similar job offers.

1. Job Offer Summary:
- Title: 
- Department: 
- Responsibilities:

2. Commonly Sought Skills and Qualifications:
- Educational Background: 
- Experience: 
- Skills: 
- Technical Proficiencies: 
- Regulatory Knowledge: 
- Certifications: 

Note: The specific requirements and preferences for qualifications may vary between job offers.

Please return the information in the JSON format.
"""
prompt = ChatPromptTemplate.from_template(template)

model = ChatOpenAI()

In [190]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [191]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"
job_offer = 'We are seeking a dynamic individual to be a part of our Audit department. As our auditor, your responsibilities will include planning and performing a variety of internal audits and special projects throughout the company, which include a variety of Senior Manage- ment driven assignments. These audits will include detailed assessment of operational, financial and systems activities where a thorough understanding of control processes must be obtained from a vari- ety of people and resources.'
answer = chain.invoke(job_offer)
response = json.loads(answer)
print(response)

{
  "job_offer_summary": {
    "title": "Auditor",
    "department": "Audit",
    "responsibilities": "Planning and performing a variety of internal audits and special projects throughout the company, including assessment of operational, financial, and systems activities."
  },
  "commonly_sought_skills_and_qualifications": {
    "educational_background": "Bachelor's degree in accounting, finance, or a related field. Professional certification such as Certified Internal Auditor (CIA) or Certified Public Accountant (CPA) is preferred.",
    "experience": "Previous experience in auditing or a related field. Knowledge of internal control processes and audit methodologies.",
    "skills": "Strong analytical and problem-solving skills. Excellent communication and interpersonal skills. Ability to work independently and in a team. Attention to detail and ability to meet deadlines.",
    "technical_proficiencies": "Proficient in using audit software and tools. Knowledge of financial systems an

In [ ]:
def analyze_job_offer():
    # Code to analyze Job OFfer
    if st.session_state.job_offer_file:
        chain = load_qa_chain(llm, chain_type="stuff")
        job_offer = st.session_state['job_offer_file']
        qry = '''**Please list generally required skills by different job offers that closely matches the given Job Offer:**
        **Job Offer Details**
        {JobOffer}
        '''
        vectordb = persist_vectorstore()
        input_query = qry.format(JobOffer=job_offer)
        context = vectordb.similarity_search(job_offer, NUM_QUERY)
        input_data = {"input_documents": context, "question": input_query}
        output = chain(input_data)
        # print("Job Offer Analyzew\n", output['output_text'])
        return output['output_text']

# Final Report

In [210]:
vectorstore = Chroma(persist_directory="ESCO_mpnet_db", embedding_function=sentence_transformer_ef)
retriever = vectorstore.as_retriever()

In [233]:
template = """
I have a curriculum vitae (CV) and a job offer. I would like the OpenAI model to compare the two and provide insights on how well the CV matches the job offer's requirements. Please use a retriever to consult the ESCO (European Skills, Competences, Qualifications and Occupations) Taxonomy to extract relevant job profile information. This will inform a more accurate comparison between the CV and the job offer.

Curriculum Vitae:
```
{cv}
```

Job Offer:
```
{job_offer}
```
Use the information from the ESCO Taxonomy to evaluate the following aspects:
- The alignment of professional experience listed in the CV with the responsibilities and projects as per ESCO job profiles related to the job offer.
- The educational qualifications in the CV compared to the educational requirements suggested by ESCO for similar jobs.
- The skills and proficiencies mentioned in the CV in comparison with those detailed in the ESCO profile for the job offer.
- Any certifications or regulatory knowledge stated in the CV that match the ESCO profile's preferences or requirements for the job offer.
- A summary of areas where the CV strongly matches the ESCO profile for the job offer and areas where there may be gaps or need for further development.

Please present the analysis in the following JSON structure:
{{
    "Match Insights": {{
        "Professional Experience Alignment": "",
        "Educational Qualifications Comparison": "",
        "Skills and Proficiencies Match": "",
        "Certifications and Regulatory Knowledge": "",
        "Overall Summary": {{
            "Strong Matches": "",
            "Potential Gaps": ""
        }}
    }}
}}

Note: The insights should be based on the retrieved information from the ESCO Taxonomy to ensure an informed and accurate analysis.
"""

template = """
I have a curriculum vitae (CV) and a job offer. I would like the OpenAI model to compare the two and provide insights on how well the CV matches the job offer's requirements. Please use a retriever to consult the ESCO (European Skills, Competences, Qualifications and Occupations) Taxonomy to extract relevant job profile information. This will inform a more accurate comparison between the CV and the job offer.

Curriculum Vitae:
```
{cv}
```

Job Offer:
```
{job_offer}
```
Instructions for the OpenAI model:
1. Retrieve job titles and profiles from the ESCO Taxonomy that align with the professional experiences mentioned in the CV.
2. Compare these ESCO job profiles with the responsibilities and requirements detailed in the job offer.
3. Provide insights on:
   - How the candidate's past job titles and experiences align with the job offer.
   - The relevance of the candidate's educational background in relation to the job offer.
   - The match between the candidate's skills and proficiencies with those required by the job offer.
   - The presence of any certifications or specific knowledge in the CV that are also mentioned in the job offer.
4. Identify any significant gaps or areas where the CV does not align with the job offer.

Return the analysis in the following JSON structure:
{{
    "ESCO Analysis": {{
        "Job Profile Match": "",
        "Educational Qualifications Comparison": "",
        "Skills and Proficiencies Match": "",
        "Certifications and Regulatory Knowledge": "",
        "Overall Summary": {{
            "Strong Matches": "",
            "Potential Gaps": ""
        }}
    }}
}}

Note: The insights should be based on the retrieved information from the ESCO Taxonomy to ensure an informed and accurate analysis.
"""

# Please return the information in the JSON format.
# """
prompt = ChatPromptTemplate.from_template(template)

chain = (
    {"context": retriever, "cv": RunnablePassthrough(), "job_offer": RunnablePassthrough()}
    | prompt 
    | model
) 
answer = chain.invoke({"cv": CV, "job_offer": job_offer})
response = json.loads(answer.content)

In [234]:
print(json.dumps(response, indent=4))

{
    "ESCO Analysis": {
        "Job Profile Match": "The candidate's past job titles and experiences align well with the job offer. The candidate has experience as a founder, sales manager, and delivery driver, which demonstrates their ability to take on different roles and responsibilities. These experiences show their skills in managing a business, leading a team, and handling logistics and distribution.",
        "Educational Qualifications Comparison": "The CV does not provide information about the candidate's educational background. Therefore, it is not possible to compare their qualifications with the job offer.",
        "Skills and Proficiencies Match": "The candidate's skills and proficiencies align with some of the requirements mentioned in the job offer. Their skills in effective time management, hard work, highly organization, fast learning, communication, ability to work in a team, and motivation are relevant to the responsibilities of an auditor. These skills demonstrat

In [235]:
print(json.dumps(response, indent=4))

{
    "ESCO Analysis": {
        "Job Profile Match": "The candidate's past job titles and experiences align well with the job offer. The candidate has experience as a founder, sales manager, and delivery driver, which demonstrates their ability to take on different roles and responsibilities. These experiences show their skills in managing a business, leading a team, and handling logistics and distribution.",
        "Educational Qualifications Comparison": "The CV does not provide information about the candidate's educational background. Therefore, it is not possible to compare their qualifications with the job offer.",
        "Skills and Proficiencies Match": "The candidate's skills and proficiencies align with some of the requirements mentioned in the job offer. Their skills in effective time management, hard work, highly organization, fast learning, communication, ability to work in a team, and motivation are relevant to the responsibilities of an auditor. These skills demonstrat

In [236]:
type(CV)

dict

In [244]:
json.loads(json.dumps(CV['Experiences'][0]))

{'job_title': 'Founder',
 'description': 'I founded a company that sells pet products and pet foods online. Our products are sold at www.zoopet.com.tr and many online marketplaces. Operations of the company involved shipment of the products to every region of Turkey. In 2022, it was taken over by an investor. The company is still inactive service today.',
 'skills': ['Ability to Work Under Pressure']}